#Cria um dataset que contem ruido em 30% de suas imagens






In [ ]:
import zipfile
import os

# Caminho para o arquivo zip
zip_path = '/content/data_val_SNR.zip'

# Caminho para o diretório onde o conteúdo será extraído
extract_dir = '/content/data'

# Criar o diretório de extração se não existir
os.makedirs(extract_dir, exist_ok=True)

# Abrir o arquivo zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extrair todos os arquivos
    zip_ref.extractall(extract_dir)

print(f"Arquivos extraídos para {extract_dir}")


In [ ]:
import cv2
import numpy as np
import os
import random
import shutil

def add_gaussian_noise(image, mean=0, sigma=15):
    noise = np.random.normal(mean, sigma, image.shape)
    noisy_image = image + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    return noisy_image

def add_gaussian_noise_to_random_region(image, mean=0, sigma=15):
    height, width, _ = image.shape
    region_height = random.randint(int(height * 0.1), int(height * 0.5))
    region_width = random.randint(int(width * 0.1), int(width * 0.5))
    top_left_x = random.randint(0, width - region_width)
    top_left_y = random.randint(0, height - region_height)
    roi = image[top_left_y:top_left_y + region_height, top_left_x:top_left_x + region_width]
    noise = np.random.normal(mean, sigma, roi.shape)
    noisy_roi = roi + noise
    noisy_roi = np.clip(noisy_roi, 0, 255).astype(np.uint8)
    image[top_left_y:top_left_y + region_height, top_left_x:top_left_x + region_width] = noisy_roi
    return image

def apply_noise_to_dataset(data_path, new_data_path, noise_probability=0.3, sigma=15):
    if not os.path.exists(new_data_path):
        os.makedirs(new_data_path)

    subsets = ['train', 'valid', 'test']
    for subset in subsets:
        images_path = os.path.join(data_path, subset, 'images')
        labels_path = os.path.join(data_path, subset, 'labels')

        new_images_path = os.path.join(new_data_path, subset, 'images')
        new_labels_path = os.path.join(new_data_path, subset, 'labels')

        os.makedirs(new_images_path, exist_ok=True)
        os.makedirs(new_labels_path, exist_ok=True)

        images = os.listdir(images_path)
        for image_name in images:
            image_path = os.path.join(images_path, image_name)
            label_path = os.path.join(labels_path, image_name.replace('.jpg', '.txt'))

            # Lê a imagem
            image = cv2.imread(image_path)

            # Decide se vai adicionar ruído
            if random.random() < noise_probability:
                # Aleatoriamente escolhe entre ruído na imagem inteira ou apenas em uma região
                if random.choice([True, False]):
                    noisy_image = add_gaussian_noise(image, sigma=sigma)
                else:
                    noisy_image = add_gaussian_noise_to_random_region(image, sigma=sigma)

                # Salva a imagem com ruído no novo dataset
                cv2.imwrite(os.path.join(new_images_path, image_name), noisy_image)
            else:
                # Copia a imagem original sem ruído
                shutil.copy(image_path, new_images_path)

            # Copia o label correspondente
            shutil.copy(label_path, new_labels_path)

# Caminho para o dataset original e o novo dataset
data_path = 'data'
new_data_path = 'data_with_noise'

# Aplica o processo
apply_noise_to_dataset(data_path, new_data_path, noise_probability=0.3, sigma=15)


In [ ]:
import shutil

# Caminho para o dataset original e o novo dataset
data_path = 'data'
new_data_path = 'data_with_noise'

# Copia os arquivos readme e data.yaml
shutil.copy(os.path.join(data_path, 'README.dataset.txt'), new_data_path)
shutil.copy(os.path.join(data_path, 'README.roboflow.txt'), new_data_path)
shutil.copy(os.path.join(data_path, 'data.yaml'), new_data_path)


In [ ]:
!zip -r /content/data_with_noise.zip /content/data_with_noise

from google.colab import files
files.download("/content/data_with_noise.zip")


  adding: content/data_with_noise/ (stored 0%)
  adding: content/data_with_noise/train/ (stored 0%)
  adding: content/data_with_noise/train/labels/ (stored 0%)
  adding: content/data_with_noise/train/labels/vlc-record-2023-11-07-18h20m16s-vlc-record-2023-10-26-07h36m35s-rtsp___192_168_1_3_8554_mystream-_avi-_mp4-41_jpg.rf.52cdfea6a1b71aa445e9444e51046101_hflip.txt (deflated 63%)
  adding: content/data_with_noise/train/labels/vlc-record-2023-11-07-18h07m47s-vlc-record-2023-10-25-12h52m23s-rtsp___192_168_1_3_8554_mystream-_avi-_mp4-34_jpg.rf.7729dc577fcce153bdbd690c9a63538f.txt (deflated 68%)
  adding: content/data_with_noise/train/labels/vlc-record-2023-11-07-18h08m49s-vlc-record-2023-10-25-12h52m23s-rtsp___192_168_1_3_8554_mystream-_avi-_mp4-2_jpg.rf.e4d25763744fc49e8d016c7ee79c4513.txt (deflated 67%)
  adding: content/data_with_noise/train/labels/vlc-record-2023-11-07-18h11m25s-vlc-record-2023-10-25-12h52m23s-rtsp___192_168_1_3_8554_mystream-_avi-_mp4-32_jpg.rf.bee9c566d2d2af24b9ccba1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>